In [1]:
# Get the API key from here: https://ai.google.dev/tutorials/setup
# Create a new secret called "GEMINI_API_KEY" via Add-ons -> Secrets in the top menu, and attach it to this notebook.
from kaggle_secrets import UserSecretsClient
from IPython.display import display
from IPython.display import Markdown

import pathlib
import textwrap

user_secrets = UserSecretsClient()
apiKey = user_secrets.get_secret("GEMINI_API_KEY")

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [2]:
import google.generativeai as genai

genai.configure(api_key = apiKey)

### Generate text from text-only inputs using `gemini-pro` ###

The `generate_content` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. The available models only support text and images as input, and text as output.

In the simplest case, you can pass a prompt string to the `GenerativeModel.generate_content` method:

In [3]:
level = input()

 Adult


In [4]:
number_of_words = input()

 600


In [7]:
how_many_errors = input()

 none


In [6]:
topic = input()

 How shakespear influences modern society


In [8]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(f"I have an assignment which should seem like it is made by a {level},  it should have atleast {number_of_words} characters and should contain {how_many_errors} errors. The topic is '{topic}.'")
to_markdown(response.text)

> William Shakespeare, the legendary playwright and poet of the Elizabethan era, has left an indelible mark on modern society, shaping cultural norms, language, and artistic expression. His influence transcends time and geographical boundaries, permeating various aspects of contemporary life.
> 
> 1. **Literary Impact:**
> 
>    - Shakespeare's literary prowess has revolutionized storytelling techniques and influenced generations of writers. His plays introduced complex characters, intricate plots, and poetic language, setting new standards for dramatic writing.
> 
> 2. **Language and Vocabulary:**
> 
>    - Shakespeare's vast vocabulary and innovative use of language have significantly enriched the English language. Many of his coined words and phrases have become an integral part of everyday speech, expanding our expressive capabilities.
> 
> 3. **Cultural and Social Commentary:**
> 
>    - Shakespeare's plays often addressed social and political issues of his time, making them relevant to modern audiences as well. His exploration of themes such as love, loss, power, and justice resonates with people across cultures and generations.
> 
> 4. **Theatrical Legacy:**
> 
>    - Shakespeare's plays have had a profound impact on the development of theater as an art form. His works continue to be performed worldwide, inspiring new interpretations and artistic adaptations that keep his stories alive for contemporary audiences.
> 
> 5. **Film and Television Adaptations:**
> 
>    - Shakespeare's plays have been adapted into countless films and television shows, reaching a vast audience beyond the traditional theater-going crowd. These adaptations have introduced Shakespeare's work to new generations and made his stories accessible to a broader spectrum of viewers.
> 
> 6. **Educational Value:**
> 
>    - Shakespeare's plays are widely studied in schools and universities around the world. His works offer valuable insights into human nature, history, and cultural heritage, making them essential texts for students of literature, drama, and history.
> 
> 7. **Cultural Icons:**
> 
>    - Shakespeare's characters have become cultural icons, recognized and referenced in popular culture. Characters like Hamlet, Romeo and Juliet, and Macbeth have transcended their literary origins and become symbols of universal human experiences.
> 
> 8. **Philosophical and Ethical Insights:**
> 
>    - Shakespeare's plays explore profound philosophical and ethical dilemmas, inviting audiences to reflect on the complexities of human existence. His works encourage critical thinking and introspection, challenging societal norms and encouraging empathy.
> 
> In conclusion, Shakespeare's influence on modern society is undeniable. His works continue to shape cultural narratives, enrich our language, and inspire artistic expression across various mediums. By exploring timeless themes, creating memorable characters, and using innovative language, Shakespeare has left an enduring legacy that continues to resonate with audiences worldwide.

In [5]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(f"How to convert .svg to syg font?")
to_markdown(response.text)

> Converting an SVG file to a SYG font is not possible. The .svg file extension represents Scalable Vector Graphics, a format used for two-dimensional graphics. On the other hand, the .syg extension is associated with a different file format, such as Synaptics Touchpad Driver or Symantec Generic License Agreement files. Therefore, there is no direct conversion process between these two file formats.

In [ ]:
# View the response candidates
response.candidates

In [ ]:
# If API failed to return a result, use prompt_feedback to see if it was blocked due to safety concerns regarding the prompt.
response.prompt_feedback

### Generate text from image and text prompts using `gemini-pro-vision`

In [ ]:
!curl -o image.jpg https://t0.gstatic.com/licensed-image?q=tbn:ANd9GcQ_Kevbk21QBRy-PgB4kQpS79brbmmEG7m3VOTShAn4PecDU5H5UxrJxE3Dw1JiaG17V88QIol19-3TM2wCHw

In [ ]:
import PIL.Image

img = PIL.Image.open('image.jpg')
img

In [ ]:
model = genai.GenerativeModel('gemini-pro-vision')

In [ ]:
response = model.generate_content(img)

to_markdown(response.text)

In [ ]:
response = model.generate_content(["Write a short, engaging blog post based on this picture. It should include a description of the meal in the photo and talk about my journey meal prepping.", img])
to_markdown(response.text)

### Chat conversations ###
Gemini enables you to have freeform conversations across multiple turns. The `ChatSession` class simplifies the process by managing the state of the conversation, so unlike with `generate_content`, you do not have to store the conversation history as a list.

In [ ]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

In [ ]:
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
to_markdown(response.text)

In [ ]:
chat.history

In [ ]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")
to_markdown(response.text)

In [ ]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

### Use embeddings ###

[Embedding](https://developers.google.com/machine-learning/glossary#embedding-vector) is a technique used to represent information as a list of floating point numbers in an array. With Gemini, you can represent text (words, sentences, and blocks of text) in a vectorized form, making it easier to compare and contrast embeddings. For example, two texts that share a similar subject matter or sentiment should have similar embeddings, which can be identified through mathematical comparison techniques such as cosine similarity.

Use the `embed_content` method to generate embeddings. The method handles embedding for the following tasks (`task_type`):

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting. Using this task type requires a `title`.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

The following generates an embedding for a single string for document retrieval:

In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string")

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED]')

Note: The `retrieval_document` task type is the only task that accepts a title.

To handle batches of strings, pass a list of strings in `content`:

In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content=[
      'What is the meaning of life?',
      'How much wood would a woodchuck chuck?',
      'How does the brain work?'],
    task_type="retrieval_document",
    title="Embedding of list of strings")

# A list of inputs > A list of vectors output
for v in result['embedding']:
  print(str(v)[:50], '... TRIMMED ...')

While the `genai.embed_content` function accepts simple strings or lists of strings, it is actually built around the `glm.Content` type (like `GenerativeModel.generate_content`). `glm.Content` objects are the primary units of conversation in the API.

While the `glm.Content` object is multimodal, the `embed_content` method only supports text embeddings. This design gives the API the *possibility* to expand to multimodal embeddings.

In [ ]:
response.candidates[0].content

In [ ]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = response.candidates[0].content)

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED ...')

In [ ]:
chat.history

In [ ]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = chat.history)

# 1 input > 1 vector output
for i,v in enumerate(result['embedding']):
  print(str(v)[:50], '... TRIMMED...')

### Encode messages
The previous sections relied on the SDK to make it easy for you to send prompts to the API. This section offers a fully-typed equivalent to the previous example, so you can better understand the lower-level details regarding how the SDK encodes messages.

Underlying the Python SDK is the `google.ai.generativelanguage` client library:

In [ ]:
import google.ai.generativelanguage as glm

The SDK attempts to convert your message to a `glm.Content` object, which contains a list of `glm.Part` objects that each contain either:

1. a `text` (string)
2. `inline_data` (`glm.Blob`), where a blob contains binary `data` and a `mime_type`.

You can also pass any of these classes as an equivalent dictionary.

Note: The only accepted mime types are some image types, `image/*`.

So, the fully-typed equivalent to the previous example is:  

In [ ]:
model = genai.GenerativeModel('gemini-pro-vision')
response = model.generate_content(
    glm.Content(
        parts = [
            glm.Part(text="Write a short, engaging blog post based on this picture."),
            glm.Part(
                inline_data=glm.Blob(
                    mime_type='image/jpeg',
                    data=pathlib.Path('image.jpg').read_bytes()
                )
            ),
        ],
    )
)
to_markdown(response.text[:100] + "... [TRIMMED] ...")

### Multi-turn conversations ###
While the `genai.ChatSession` class shown earlier can handle many use cases, it does make some assumptions. If your use case doesn't fit into this chat implementation it's good to remember that `genai.ChatSession` is just a wrapper around `GenerativeModel.generate_content`. In addition to single requests, it can handle multi-turn conversations.

The individual messages are `glm.Content` objects or compatible dictionaries, as seen in previous sections. As a dictionary, the message requires `role` and `parts` keys. The `role` in a conversation can either be the `user`, which provides the prompts, or `model`, which provides the responses.

Pass a list of `glm.Content` objects and it will be treated as multi-turn chat:

In [ ]:
model = genai.GenerativeModel('gemini-pro')

messages = [
    {'role':'user',
     'parts': ["Briefly explain how a computer works to a young child."]}
]
response = model.generate_content(messages)

to_markdown(response.text)

To continue the conversation, add the response and another message.

Note: For multi-turn conversations, you need to send the whole conversation history with each request. The API is **stateless**.

In [ ]:
messages.append({'role':'model',
                 'parts':[response.text]})

messages.append({'role':'user',
                 'parts':["Okay, how about a more detailed explanation to a high school student?"]})

response = model.generate_content(messages)

to_markdown(response.text)

### Generation configuration

The `generation_config` argument allows you to modify the generation parameters. Every prompt you send to the model includes parameter values that control how the model generates responses.

In [ ]:
response = model.generate_content(
    'Tell me a story about a magic backpack.',
    generation_config=genai.types.GenerationConfig(
        # Only one candidate for now.
        candidate_count=1,
        stop_sequences=['x'],
        max_output_tokens=20,
        temperature=1.0)
)